# Simple test with real data

In [ ]:
import csv
import itertools as itt

import matplotlib.pyplot as plt
import numpy as np
import length_priors

import dynamic_computation as dc
import readers
import segment_viz

In [ ]:
from default_priors import arcPriorTempo, arcPriorLoud, lengthPriorParamsTempo as lengthPriorParams

# Loading some data

In [ ]:
timingsData = readers.readAllMazurkaTimingsAndSeg(timingPath="data/beat_time", segPath="data/deaf_structure_tempo")
dynData = readers.readAllMazurkaDataAndSeg(timingPath="data/beat_dyn", segPath="data/deaf_structure_loudness")

fullData = [(piece, interpret, tempo, tempoSeg, dyn, dynSeg) 
            for ((piece, interpret, tempo, tempoSeg),(piece2, interpret2, dyn, dynSeg))
            in itt.product(timingsData,dynData)
            if interpret == interpret2]

In [ ]:
#arcPrior = [arcPriorTempo, arcPriorLoud]
arcPrior = arcPriorTempo

# Get it Running

In [ ]:
# Unpack the data
(piece, interpret, tempo, tempoSeg, dyn, dynSeg) = fullData[0]

piece_formatted = piece[16:20]
print(piece_formatted, interpret)

#sampleData = list(zip(tempo, dyn[1:]))
sampleData, times = tempo
sampleData = list(sampleData)
segs = (tempoSeg, dynSeg)

tatums = list(range(len(sampleData)))

# Manually correct large anomalies
sampleData[106] = 200
sampleData[195] = 200

## Using a Gaussian Prior

In [ ]:

length_prior = length_priors.NormalLengthPrior(lengthPriorParams['mean'], lengthPriorParams['stddev'], range(
    len(sampleData)), lengthPriorParams['maxLength'])

post_boundaries, post_segments = dc.runAlphaBeta(sampleData, arcPrior, length_prior, return2D=True)

In [ ]:
_ = segment_viz.plot_segment_with_signal(post_marginals=post_boundaries, data=sampleData, data_time=list(range(len(sampleData))), smoothing=1)
_ = segment_viz.plot_segment_beams(post2_bidim=post_segments, max_length=length_prior.maxLength)

## Using a Geometric Prior

In [ ]:

length_prior = length_priors.GeometricLengthPrior(len(sampleData), 0.95, min_length=5, max_length=30)

post_boundaries, post_segments = dc.runAlphaBeta(sampleData, arcPrior, length_prior, return2D=True)
# _ = segment_viz.plot_segment_with_signal(post_marginals=post_boundaries, data=sampleData, data_time=sampleData)
# _ = segment_viz.plot_segment_beams(post2_bidim=post_segments, length_prior=length_prior)

In [ ]:
_ = segment_viz.plot_segment_with_signal(post_marginals=post_boundaries, data=sampleData, data_time=list(range(len(sampleData))), smoothing=1)
_ = segment_viz.plot_segment_beams(post2_bidim=post_segments, max_length=length_prior.maxLength)

## Using an Empirical Prior

In [ ]:
length_observations = sum([[length for length in np.diff(t_seg)] 
                           for (_piece, _interpret, _tempo, t_seg, _dyn, _dynSeg) in fullData],
                          start=[])

In [ ]:
length_prior = length_priors.EmpiricalLengthPrior(length_observations, len(sampleData), maxLength=30)

post_boundaries, post_segments = dc.runAlphaBeta(sampleData, arcPrior, length_prior, return2D=True)
# _ = segment_viz.plot_segment_with_signal(post_marginals=post_boundaries, data=sampleData, data_time=sampleData)
# _ = segment_viz.plot_segment_beams(post2_bidim=post_segments, length_prior=length_prior)

In [ ]:
_ = segment_viz.plot_segment_with_signal(post_marginals=post_boundaries, data=sampleData, data_time=list(range(len(sampleData))), smoothing=1)
_ = segment_viz.plot_segment_beams(post2_bidim=post_segments, max_length=length_prior.maxLength)

In [ ]:
import pandas as pd
times = np.cumsum(1/(tempo/60))+2
df = pd.DataFrame.from_dict({'time':times, 'p_boundary': post_boundaries})
df.to_csv("boundaries_post.csv")

## Using a Uniform Prior

In [ ]:

length_prior4 = length_priors.EmpiricalLengthPrior(range(5, 30), len(sampleData), maxLength=30)

post_boundaries4, post_segments4 = dc.runAlphaBeta(sampleData, arcPrior, length_prior4, return2D=True)
# _ = segment_viz.plot_segment_with_signal(post_marginals=post_boundaries, data=sampleData, data_time=sampleData)
# _ = segment_viz.plot_segment_beams(post2_bidim=post_segments, length_prior=length_prior)

In [ ]:
_ = segment_viz.plot_segment_with_signal(post_marginals=post_boundaries4, data=sampleData, data_time=list(range(len(sampleData))), smoothing=1)
_ = segment_viz.plot_segment_beams(post2_bidim=post_segments4, max_length=length_prior4.maxLength)